In [ ]:
import os
os.chdir('/home/mmr/DUQ')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR

import matplotlib.pyplot as plt
import numpy as np
import ipdb

from src.model.backbone import Backbone
from src.utils.utils import train_standard, test_standard

In [ ]:
batch_size  = 100

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.286,), (0.353,))
        ])

data_root = '~/DUQ/data/fashion_mnist/'

train_dataset = torchvision.datasets.FashionMNIST(data_root,transform=transform)
train_loader  = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size)

test_dataset  = torchvision.datasets.FashionMNIST(data_root,transform=transform,train=False)
test_loader   = torch.utils.data.DataLoader(test_dataset,batch_size = batch_size)


In [ ]:
backbone = Backbone(256).cuda()
celoss   = nn.CrossEntropyLoss(reduction='mean')
opt      = optim.SGD(backbone.parameters(),lr = 0.05,momentum=0.9,weight_decay = 1e-4)
sched    = StepLR(opt,step_size=10,gamma = 0.2)

In [ ]:
for epoch in range(30):
    train_standard(backbone,celoss,epoch,'cuda',train_loader,opt)
    test_standard(backbone,celoss,epoch,batch_size,'cuda',test_loader)
    sched.step()